In [19]:
import numpy as np
import meshio
import meshplot as mp

In [20]:
def tet_volume(a, b, c, d):
    return np.cross(b - a, c - a).dot(d - a) / 6.0

In [21]:
def hexes2tets(hexes):
    # reference cube:
    # 0: -0.5 -0.5 -0.5
    # 1: 0.5 -0.5 -0.5
    # 2: 0.5 -0.5 0.5
    # 3: -0.5 -0.5 0.5
    # 4: -0.5 0.5 -0.5
    # 5: 0.5 0.5 -0.5
    # 6: 0.5 0.5 0.5
    # 7: -0.5 0.5 0.5

    hex2tet = np.array([
        [0, 4, 6, 5],
        [3, 6, 2, 0],
        [0, 4, 7, 6],
        [3, 6, 0, 7],
        [2, 0, 6, 1],
        [6, 0, 5, 1],
    ])

    reorder = np.array([6, 2, 0, 4, 7, 3, 1, 5])

    tets = []
    for hex in hexes:
        for tet in hex2tet:
            tets.append(hex[reorder][tet][[0, 1, 3, 2]])
    return np.asarray(tets)

In [22]:
def print_stats(points, tets):
    vols = [tet_volume(*points[tet]) for tet in tets]
    print(min(vols), max(vols), sum(vols))

In [23]:
LENGTH=5
DIAMETER=LENGTH/2.5
RADIUS=DIAMETER/2
THICKNESS=0.01
THETA_N=100
side_length = np.sqrt(2 * RADIUS**2 * (1 - np.cos(2 * np.pi / THETA_N)))
Z_N=int(np.round(LENGTH / side_length))

In [24]:
theta = np.linspace(0, 2*np.pi, THETA_N, endpoint=False).reshape(-1, 1)
outer_points = np.hstack([
    RADIUS * np.cos(theta), RADIUS * np.sin(theta), np.zeros_like(theta)
])
inner_points = (RADIUS - THICKNESS)/RADIUS * outer_points
zs = np.linspace(-LENGTH/2, LENGTH/2, Z_N)
points = []
for z in zs:
    points.append(outer_points + np.array([0, 0, z]))
    points.append(inner_points + np.array([0, 0, z]))
points = np.vstack(points)

In [25]:
hexes = []
points_per_z = 2 * THETA_N
for zi in range(Z_N-1):
    z_offset = points_per_z * zi
    for i in range(THETA_N):
        hexes.append([
            # outter ring
            i + z_offset,
            (i+1) % THETA_N + z_offset,
            i + points_per_z + z_offset,
            (i+1) % THETA_N + points_per_z + z_offset,
            # inner ring
            i + THETA_N + z_offset,
            (i+1) % THETA_N + THETA_N + z_offset,
            i + THETA_N + points_per_z + z_offset,
            (i+1) % THETA_N + THETA_N + points_per_z + z_offset,
        ])
hexes = np.asarray(hexes)

In [26]:
tets = hexes2tets(hexes)
print_stats(points, tets)

6.557237798947155e-06 6.6234725241893034e-06 0.3123828346583093


In [27]:
mp.plot(points, tets, shading={"point_size": 0.1, "wireframe": True})
meshio.Mesh(points, {"tetra": tets}).write(f"../meshes/hollow-cylinder/N={THETA_N:d}.msh", file_format="gmsh")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…